# 문제 6

[Kaggle 형] train_prob.csv로 문제 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


# Kaggle Tip

**Pain point** 

> 데이터처리 과정을 Train에서 학습하고 변환 결과를 Train과 Test 데이터프레임에 적용하거나 생성

> 여러 처리기를 변수별로 따로 적용

**데이터처리 과정와 모델 학습을 한 번에**

=> sklearn.compsose.ColumnTransformer + sklearn.pipeline.make_pipeline

> ColumnTransformer : DataFrame에 컬럼별 처리기(StandardScaler, OneHotEncoder, PCA, ...) 적용

> make_pipeline: 단계화된 처리 모델 구성


Ex) 

cat0, cat1는 가변수, cont0, cont1: 표준화 , na_1, na_2: 통과

위 처리를 거친후 대상 변수 failure를  로직스틱 회귀 모델로 학습 


```python
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from skelarn.pipeline import make_pipeline

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), ['cat0', 'cat1']), #  cat0, cat1은 OneHotEncoder를 적용
    ('std', StandardScaler(), ['cont0', 'cont1']), #  cont0, cont1은 StandardScaler를 적용
    ('pt', 'passthrough', ['na_1', 'na_2']) # na_1, na_2는 통과 
], remainder='drop') # remainder: 'drop' 나머지는 제외, 'passthrough'는 나머지 통과
X = ['cat0', 'cat1', 'cont0', 'cont1', 'na_1', 'na_2']
clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs')) # ColumnTransformer->LogisticRegression pipeline 구성
clf_lr.fit(df[X], df['failure'])
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id') # 실제 시험에서는 주어지지 않습니다.

In [3]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [4]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [5]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
imp = IterativeImputer(
    estimator=LinearRegression(),
    random_state=123
)
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']

df_train = df_train.assign(
    na_1=lambda x: x['measurement_3'].isna(),
    na_2=lambda x: x['measurement_5'].isna(),
)

df_test = df_test.assign(
    na_1=lambda x: x['measurement_3'].isna(),
    na_2=lambda x: x['measurement_5'].isna(),
)

# Train에 등장했던 product_code 범주값이 테스트에는 등장하지 않아 각각 따로 모델을 구성(fit_transform)을 하게 됩니다.
# 일반적으로는 train에 했던 내용을 test 그대로 적용하지만 데이터의 특성상 이렇게 처리하는 것이니,
# 꼭 염두해주세요.
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)

na_mean_cols = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[na_mean_cols] = df_train.groupby('product_code')[na_mean_cols].transform(lambda x: x.fillna(x.mean()))
df_test[na_mean_cols] = df_test.groupby('product_code')[na_mean_cols].transform(lambda x: x.fillna(x.mean()))

df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
# loading은 product_code로 구분되는 것은 아니니, train을 사용합니다.
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean()) 

In [6]:
# 문제 3의 내용을 뽑아와서 베이스라인 모델을 만들어 냅니다.
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# 문제 3의 속성(입력변수) 선택의 최종결과만 취합니다.
X_model_1 = ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1']

ct_model_1 = ColumnTransformer([
    ('std', StandardScaler(), X_model_1[:-1]),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr = make_pipeline(ct_model_1, LogisticRegression(solver='lbfgs'))
clf_lr.fit(df_train[X_model_1], df_train['failure'])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('std',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['loading', 'measurement_1',
                                                   'measurement_4',
                                                   'measurement_14',
                                                   'measurement_17']),
                                                 ('pt', 'passthrough',
                                                  ['na_1'])],
                                   verbos

In [7]:
# answer6.csv에 test 예측 결과를 출력합니다.
pd.DataFrame({'target': clf_lr.predict_proba(df_test[X_model_1])[:, 1]}, index=df_test.index)\
    .to_csv('answer6.csv')

In [8]:
# 자가 체점을 해봅니다.
from sklearn.metrics import roc_auc_score
roc_auc_score(df_ans['failure'], clf_lr.predict_proba(df_test[X_model_1])[:, 1])

0.5883988309352517